In [ ]:
!pip install  opencv-python==3.4.2.17
!pip install  opencv-contrib-python==3.4.2.17

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.0 MB 1.5 MB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 30.6 MB 59 kB/s 
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


Importing Liraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import cv2

In [ ]:
import os
import csv

In [ ]:
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score

In [ ]:
import pickle

In [ ]:
from sklearn.metrics import confusion_matrix          # to get confusion matrix for testing data
from datetime import datetime                         # to calculate time
from sklearn import linear_model  

# **Data folders**

In [ ]:
folder1 = "/content/drive/MyDrive/CVCP_ID_12_Final/with_helmet"
folder2 = "/content/drive/MyDrive/CVCP_ID_12_Final/without_helmet"

# Sift

In [ ]:
i=0
for filename in os.listdir(folder1):
    #path
    path=os.path.join(folder1,filename)
    a=cv2.imread(path)
    
    #resize image
    resize=(150,150)
    img=cv2.resize(a,resize)
    
    #gray image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #initialise sift descriptor
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    
    #convert the descriptor array into a dataframe format
    out=pd.DataFrame(descriptors)
    print("descriptor shape ",i," : ", out.shape)
    i=i+1
    
    #drop first coloumn as it's the no of feature detected. Not required.
    #append to the csv file
    csv_data=out.to_csv(r'/content/drive/MyDrive/CVCP_ID_12_Final/helmet_data.csv', mode='a', header=False,index=False)

Streaming output truncated to the last 5000 lines.
descriptor shape  2000  :  (346, 128)
descriptor shape  2001  :  (305, 128)
descriptor shape  2002  :  (245, 128)
descriptor shape  2003  :  (342, 128)
descriptor shape  2004  :  (307, 128)
descriptor shape  2005  :  (285, 128)
descriptor shape  2006  :  (384, 128)
descriptor shape  2007  :  (281, 128)
descriptor shape  2008  :  (277, 128)
descriptor shape  2009  :  (296, 128)
descriptor shape  2010  :  (249, 128)
descriptor shape  2011  :  (245, 128)
descriptor shape  2012  :  (198, 128)
descriptor shape  2013  :  (499, 128)
descriptor shape  2014  :  (299, 128)
descriptor shape  2015  :  (500, 128)
descriptor shape  2016  :  (480, 128)
descriptor shape  2017  :  (227, 128)
descriptor shape  2018  :  (365, 128)
descriptor shape  2019  :  (444, 128)
descriptor shape  2020  :  (212, 128)
descriptor shape  2021  :  (521, 128)
descriptor shape  2022  :  (436, 128)
descriptor shape  2023  :  (447, 128)
descriptor shape  2024  :  (314, 128)

In [ ]:
data1 = pd.read_csv(r'/content/drive/MyDrive/CVCP_ID_12_Final/helmet_data.csv',header=None,dtype='uint8')
#MemoryError: Unable to allocate 166. MiB for an array with shape (128, 169606) and data type float64,
#Double-precision floating-point format

data1=data1.astype(np.uint8) 
#as unit8 contain 1 byte(8bit) and ranges from 0 to 255
data1

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,76,47,61,14,23,7,1,33,9,24,...,3,4,23,74,61,10,32,67,7,3
1,0,0,0,0,0,0,0,0,2,0,...,40,12,8,22,43,56,34,7,7,3
2,0,0,0,0,0,0,0,0,0,3,...,20,43,125,69,53,26,5,11,33,53
3,94,22,2,1,43,39,4,15,127,112,...,0,0,0,0,0,0,0,0,0,0
4,86,106,56,67,25,4,1,1,133,133,...,0,1,51,74,84,15,13,13,7,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562400,4,5,4,8,31,38,18,9,51,15,...,2,4,18,16,3,41,53,17,6,1
1562401,0,5,20,27,6,0,0,0,4,14,...,12,11,14,23,7,15,74,54,2,1
1562402,33,26,11,8,60,34,4,10,101,76,...,39,27,3,0,0,0,0,15,46,60
1562403,29,1,1,9,108,1,0,5,132,51,...,5,88,46,0,0,0,5,2,1,30


In [ ]:
i=1
for filename in os.listdir(folder2):
    #path
    path=os.path.join(folder2,filename)
    b=cv2.imread(path)
    
    #resize image
    resize=(150,150)
    img=cv2.resize(b,resize)
    
    #gray image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #initialise sift descriptor
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    
    #convert the descriptor array into a dataframe format
    out=pd.DataFrame(descriptors)
    i=i+1
    
    #drop first coloumn as it's the no of feature detected. Not required.
    #append to the csv file
    csv_data=out.to_csv(r'/content/drive/MyDrive/CVCP_ID_12_Final/nohelmet_data.csv', mode='a', header=False,index=False)

In [ ]:
data2= pd.read_csv(r'/content/drive/MyDrive/CVCP_ID_12_Final/nohelmet_data.csv',header=None,dtype='uint8')
data2=data2.astype(np.uint8)
data2

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,14,17,40,13,18,46,11,5,36,14,...,0,2,2,0,0,0,9,4,1,0
1,13,5,13,9,0,0,0,0,15,10,...,1,14,2,8,11,90,77,12,7,1
2,0,0,0,0,0,17,8,0,5,0,...,0,0,0,21,24,35,145,18,0,0
3,111,20,1,8,7,12,14,25,124,89,...,4,1,0,3,42,6,3,1,0,0
4,11,28,100,73,32,2,0,0,139,81,...,28,57,0,0,18,9,5,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1549011,4,10,1,0,0,0,0,0,19,4,...,0,3,134,59,2,0,0,0,7,69
1549012,54,2,0,0,0,0,0,71,132,16,...,4,132,36,18,3,0,0,0,0,9
1549013,21,0,0,0,0,1,45,47,129,31,...,1,22,38,94,7,0,0,0,0,1
1549014,2,126,98,28,1,0,0,0,7,133,...,0,1,1,8,1,0,0,0,0,0


In [ ]:
data=data1.append(data2)

In [ ]:
csv_data=data.to_csv(r'/content/drive/MyDrive/CVCP_ID_12_Final/detect_helmet1.csv', mode='a', header=False,index=False)

In [ ]:
data= pd.read_csv(r'/content/drive/MyDrive/CVCP_ID_12_Final/detect_helmet1.csv',header=None,dtype='uint8')
data=data.astype(np.uint8)
data

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,76,47,61,14,23,7,1,33,9,24,...,3,4,23,74,61,10,32,67,7,3
1,0,0,0,0,0,0,0,0,2,0,...,40,12,8,22,43,56,34,7,7,3
2,0,0,0,0,0,0,0,0,0,3,...,20,43,125,69,53,26,5,11,33,53
3,94,22,2,1,43,39,4,15,127,112,...,0,0,0,0,0,0,0,0,0,0
4,86,106,56,67,25,4,1,1,133,133,...,0,1,51,74,84,15,13,13,7,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3111416,4,10,1,0,0,0,0,0,19,4,...,0,3,134,59,2,0,0,0,7,69
3111417,54,2,0,0,0,0,0,71,132,16,...,4,132,36,18,3,0,0,0,0,9
3111418,21,0,0,0,0,1,45,47,129,31,...,1,22,38,94,7,0,0,0,0,1
3111419,2,126,98,28,1,0,0,0,7,133,...,0,1,1,8,1,0,0,0,0,0


In [ ]:
import sklearn.cluster as cluster
import seaborn as sns 

In [ ]:
import sklearn.metrics as metrics
for i in range(2,15):
    labels=cluster.KMeans(n_clusters=i,init="k-means++",random_state=200).fit(data).labels_
    print ("Silhouette score for k(clusters) = "+str(i)+" is " +str(metrics.silhouette_score(data,labels,metric="euclidean",sample_size=1000,random_state=200)))

Silhouette score for k(clusters) = 2 is 0.08190853511318358
Silhouette score for k(clusters) = 3 is 0.06036493285720827


KeyboardInterrupt: ignored

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 2, in where
KeyboardInterrupt


Silhouette score for k(clusters) = 4 is 0.05285052423371788


In [ ]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(data)

KMeans(n_clusters=5)

In [ ]:
# save the model to disk
filename = 'Kmeans_helmet_Model.sav'
pickle.dump(kmeans, open(filename, 'wb'))

In [ ]:
hist=np.histogram(kmeans.labels_,bins=[0,1,2,3,4,5])
print('histogram of trained kmeans')
print(hist,"\n")

histogram of trained kmeans
(array([563289, 762158, 673193, 544775, 568006]), array([0, 1, 2, 3, 4, 5])) 



In [ ]:
i=0
data=[]
#k=0

for filename in os.listdir(folder1):
    #path
    path=os.path.join(folder1,filename)
    a=cv2.imread(path)
    
    #resize image
    resize=(150,150)
    print(path)
    img=cv2.resize(a,resize)
    
    #gray image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    sift = cv2.xfeatures2d.SIFT_create()
    
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    out=pd.DataFrame(descriptors)
    
    #predict values of feature vector with pretrained kmeans
    #ValueError: Buffer dtype mismatch, expected 'float', in order to avoid this dtype=np.double
    
    array_double = np.array(out, dtype=np.double)
    a=kmeans.predict(array_double)
    
    #hist= np.histogram(a,bins=[0,1,2,3,4,5,6,7,8])
    hist= np.histogram(a,bins=[0,1,2,3,4,5])
    #append the dataframe into the array in append mode, the array will only have 5 values which will store the values in a row
    data.append(hist[0])
    #k=k+1
    
#convert Array to Dataframe and append to the list
Output = pd.DataFrame(data)
#add row class 
Output["Class"] = i 
csv_data=Output.to_csv(r'/content/drive/MyDrive/CVCP_ID_12_Final/final_helmet1.csv', mode='a',header=False,index=False)

Streaming output truncated to the last 5000 lines.
/content/drive/MyDrive/CVCP_ID_12_Final/with_helmet/3991.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/with_helmet/6289.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/with_helmet/3125.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/with_helmet/3329.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/with_helmet/6784.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/with_helmet/3158.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/with_helmet/4383.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/with_helmet/2425.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/with_helmet/3553.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/with_helmet/1986.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/with_helmet/2019.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/with_helmet/1952.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/with_helmet/2662.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/with_helmet/1850.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/with_helmet/2055.jpg
/content/drive/MyDrive/CVCP_ID_12_

In [ ]:
i=1
data=[]
#k=0

for filename in os.listdir(folder2):
    #path
    path=os.path.join(folder2,filename)
    a=cv2.imread(path)
    
    #resize image
    resize=(150,150)
    print(path)
    img=cv2.resize(a,resize)
    
    #gray image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    sift = cv2.xfeatures2d.SIFT_create()
    
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    out=pd.DataFrame(descriptors)
    
    #predict values of feature vector with pretrained kmeans
    #ValueError: Buffer dtype mismatch, expected 'float', in order to avoid this dtype=np.double
    
    array_double = np.array(out, dtype=np.double)
    a=kmeans.predict(array_double)
    
    #hist= np.histogram(a,bins=[0,1,2,3,4,5,6,7,8])
    hist= np.histogram(a,bins=[0,1,2,3,4,5])
    #append the dataframe into the array in append mode, the array will only have 5 values which will store the values in a row
    data.append(hist[0])
    #k=k+1
    
#convert Array to Dataframe and append to the list
Output = pd.DataFrame(data)
#add row class 
Output["Class"] = i 
csv_data=Output.to_csv(r'/content/drive/MyDrive/CVCP_ID_12_Final/final_nohelmet1.csv', mode='a',header=False,index=False)

Streaming output truncated to the last 5000 lines.
/content/drive/MyDrive/CVCP_ID_12_Final/without_helmet/3473.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/without_helmet/3701.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/without_helmet/3703.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/without_helmet/3705.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/without_helmet/3704.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/without_helmet/3706.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/without_helmet/3709.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/without_helmet/3716.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/without_helmet/3717.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/without_helmet/3718.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/without_helmet/3719.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/without_helmet/3721.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/without_helmet/2721.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/without_helmet/5521.jpg
/content/drive/MyDrive/CVCP_ID_12_Final/without_helme

In [ ]:
dat1= pd.read_csv(r'/content/drive/MyDrive/CVCP_ID_12_Final/final_helmet1.csv',header=None)
print(dat1)

       0   1   2   3    4  5
0     58  47  49  22   69  0
1     23  24  28  42   26  0
2     43  58  27  48   50  0
3     50  45  29  37   57  0
4     54  54  49  86   71  0
...   ..  ..  ..  ..  ... ..
6983  50  76  54  26   96  0
6984  40  38  35  63   64  0
6985  28  25  27  35   29  0
6986  49  44  55  39   70  0
6987  70  95  70  39  112  0

[6988 rows x 6 columns]


In [ ]:
dat2= pd.read_csv(r'/content/drive/MyDrive/CVCP_ID_12_Final/final_nohelmet1.csv',header=None)
print(dat2)

       0    1    2   3   4  5
0     81  107  109  84  42  1
1     28   34   38  29  42  1
2     38   58   47  39  56  1
3     43   58   64  28  39  1
4     61   73   93  68  47  1
...   ..  ...  ...  ..  .. ..
6983  64   80   73  58  43  1
6984  40   73   61  50  39  1
6985  54   85   72  86  46  1
6986  68   86   84  53  36  1
6987  52   62   59  62  38  1

[6988 rows x 6 columns]


In [ ]:
A=dat1.append(dat2)
csv_data=A.to_csv(r'C:\Users\CEPlus\Desktop\TYSEM2\ComputerVision\CVCP_dataset\1_classifier_lane\classifier1.csv', mode='a',header=False,index=False)

In [ ]:
A

,0,1,2,3,4,5
0,58,47,49,22,69,0
1,23,24,28,42,26,0
2,43,58,27,48,50,0
3,50,45,29,37,57,0
4,54,54,49,86,71,0
...,...,...,...,...,...,...
6983,64,80,73,58,43,1
6984,40,73,61,50,39,1
6985,54,85,72,86,46,1
6986,68,86,84,53,36,1


In [ ]:
x = A.iloc[:,0:5].values
x

array([[58, 47, 49, 22, 69],
       [23, 24, 28, 42, 26],
       [43, 58, 27, 48, 50],
       ...,
       [54, 85, 72, 86, 46],
       [68, 86, 84, 53, 36],
       [52, 62, 59, 62, 38]])

In [ ]:
y = A.iloc[:,5].values
y

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
from sklearn.preprocessing import StandardScaler
xs = StandardScaler()
x_pca = xs.fit_transform(x)
x_pca

array([[ 0.89491819, -0.00819021,  0.06554729, -1.10918959,  1.14552928],
       [-0.84317616, -1.03339485, -0.83023716,  0.12217134, -0.78949758],
       [ 0.15002061,  0.48212506, -0.87289356,  0.49157962,  0.29051741],
       ...,
       [ 0.69627884,  1.68562616,  1.04664454,  2.83116538,  0.11051491],
       [ 1.39151658,  1.73020027,  1.55852137,  0.79941985, -0.33949133],
       [ 0.59695916,  0.66042152,  0.49211131,  1.35353227, -0.24949008]])

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=None)
pca.fit(x_pca)

PCA()

In [ ]:
cycle_pca = pca.transform(x_pca)
cycle_pca

array([[ 0.54082855,  1.60251945, -0.36079255,  0.20901604, -0.5597838 ],
       [-1.60391343, -0.69661058, -0.16256096,  0.00707661, -0.14864798],
       [ 0.23647319, -0.31717036, -0.82739724,  0.62383772,  0.3356613 ],
       ...,
       [ 2.72612109, -2.17922744,  0.05162382, -0.06033455,  0.51209555],
       [ 2.4303489 , -0.4001589 ,  1.32228246,  0.38214634,  0.38991926],
       [ 1.23109757, -1.1351911 ,  0.27523627,  0.21864868,  0.0341019 ]])

In [ ]:
cycle_pca = pd.DataFrame(cycle_pca)
cycle_pca

,0,1,2,3,4
0,0.540829,1.602519,-0.360793,0.209016,-0.559784
1,-1.603913,-0.696611,-0.162561,0.007077,-0.148648
2,0.236473,-0.317170,-0.827397,0.623838,0.335661
3,0.028128,0.432263,-0.801349,0.594508,-0.308693
4,2.013422,-1.890845,-1.434289,-0.165613,-0.640055
...,...,...,...,...,...
13971,2.218247,-0.629296,0.680872,0.353539,0.265705
13972,0.996957,-0.506805,0.405054,-0.064132,0.811387
13973,2.726121,-2.179227,0.051624,-0.060335,0.512096
13974,2.430349,-0.400159,1.322282,0.382146,0.389919


In [ ]:
cycle_pca.shape

(13976, 5)

In [ ]:
print(pca.explained_variance_ratio_) 

[0.60821843 0.14744735 0.1261277  0.08498407 0.03322244]


In [ ]:
pca = PCA(n_components=5)
pca.fit(x_pca)

PCA(n_components=5)

In [ ]:
# save the model to disk
filename = 'PCA_helmet_Model.sav'
pickle.dump(kmeans, open(filename, 'wb'))

In [ ]:
cycle_pca = pca.transform(x_pca)
cycle_pca

array([[ 0.54082855,  1.60251945, -0.36079255,  0.20901604, -0.5597838 ],
       [-1.60391343, -0.69661058, -0.16256096,  0.00707661, -0.14864798],
       [ 0.23647319, -0.31717036, -0.82739724,  0.62383772,  0.3356613 ],
       ...,
       [ 2.72612109, -2.17922744,  0.05162382, -0.06033455,  0.51209555],
       [ 2.4303489 , -0.4001589 ,  1.32228246,  0.38214634,  0.38991926],
       [ 1.23109757, -1.1351911 ,  0.27523627,  0.21864868,  0.0341019 ]])

In [ ]:
cycle_pca.shape

(13976, 5)

In [ ]:
print(pca.explained_variance_ratio_) 

[0.60821843 0.14744735 0.1261277  0.08498407 0.03322244]


In [ ]:
cycle_pca = pd.DataFrame(cycle_pca)
cycle_pca

,0,1,2,3,4
0,0.540829,1.602519,-0.360793,0.209016,-0.559784
1,-1.603913,-0.696611,-0.162561,0.007077,-0.148648
2,0.236473,-0.317170,-0.827397,0.623838,0.335661
3,0.028128,0.432263,-0.801349,0.594508,-0.308693
4,2.013422,-1.890845,-1.434289,-0.165613,-0.640055
...,...,...,...,...,...
13971,2.218247,-0.629296,0.680872,0.353539,0.265705
13972,0.996957,-0.506805,0.405054,-0.064132,0.811387
13973,2.726121,-2.179227,0.051624,-0.060335,0.512096
13974,2.430349,-0.400159,1.322282,0.382146,0.389919


In [ ]:
B=pd.concat([cycle_pca, pd.DataFrame(y)],axis=1)
B

,0,1,2,3,4,0
0,0.540829,1.602519,-0.360793,0.209016,-0.559784,0
1,-1.603913,-0.696611,-0.162561,0.007077,-0.148648,0
2,0.236473,-0.317170,-0.827397,0.623838,0.335661,0
3,0.028128,0.432263,-0.801349,0.594508,-0.308693,0
4,2.013422,-1.890845,-1.434289,-0.165613,-0.640055,0
...,...,...,...,...,...,...
13971,2.218247,-0.629296,0.680872,0.353539,0.265705,1
13972,0.996957,-0.506805,0.405054,-0.064132,0.811387,1
13973,2.726121,-2.179227,0.051624,-0.060335,0.512096,1
13974,2.430349,-0.400159,1.322282,0.382146,0.389919,1


In [ ]:
csv_data=B.to_csv(r'/content/drive/MyDrive/CVCP_ID_12_Final/FinalPCAClassifier1.csv', mode='a',header=False,index=False)

In [ ]:
data= pd.read_csv(r'/content/drive/MyDrive/CVCP_ID_12_Final/FinalPCAClassifier1.csv',header=None)
data

,0,1,2,3,4,5
0,0.540829,1.602519,-0.360793,0.209016,-0.559784,0
1,-1.603913,-0.696611,-0.162561,0.007077,-0.148648,0
2,0.236473,-0.317170,-0.827397,0.623838,0.335661,0
3,0.028128,0.432263,-0.801349,0.594508,-0.308693,0
4,2.013422,-1.890845,-1.434289,-0.165613,-0.640055,0
...,...,...,...,...,...,...
13971,2.218247,-0.629296,0.680872,0.353539,0.265705,1
13972,0.996957,-0.506805,0.405054,-0.064132,0.811387,1
13973,2.726121,-2.179227,0.051624,-0.060335,0.512096,1
13974,2.430349,-0.400159,1.322282,0.382146,0.389919,1


In [ ]:
#assigning x the columns from 1 to 128 for training
x = data.iloc[:,0:5].values
print("X values")
print(x)

#assigning y with the column "Class" as target variable
y = data.iloc[:,5]
print("Y values")
print(y)

X values
[[ 0.54082855  1.60251945 -0.36079255  0.20901604 -0.5597838 ]
 [-1.60391343 -0.69661058 -0.16256096  0.00707661 -0.14864798]
 [ 0.23647319 -0.31717036 -0.82739724  0.62383772  0.3356613 ]
 ...
 [ 2.72612109 -2.17922744  0.05162382 -0.06033455  0.51209555]
 [ 2.4303489  -0.4001589   1.32228246  0.38214634  0.38991926]
 [ 1.23109757 -1.1351911   0.27523627  0.21864868  0.0341019 ]]
Y values
0        0
1        0
2        0
3        0
4        0
        ..
13971    1
13972    1
13973    1
13974    1
13975    1
Name: 5, Length: 13976, dtype: int64


In [ ]:
#Dataset split into train and test with 80% Training and 20% Testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.20, random_state=0)

## DT

In [ ]:
#Decision Tree Classifier
#Assign model with Decision Tree classifier
train_start_time = datetime.now()
print('training the model..')
model1 = DecisionTreeClassifier(max_depth=18)
model1.fit(x_train, y_train)
TRAIN_ACCURACY= accuracy_score(y_train, model1.predict(x_train))
print('Done')
train_end_time = datetime.now()
TIMING =  train_end_time - train_start_time
print('training_time(HH:MM:SS.ms) - {}\n\n'.format(TIMING))
#predicting the traget variable using testing variables
test_start_time = datetime.now()
# predict test data
print('Predicting test data')
y_pred1 = model1.predict(x_test)
TESTING_ACCURACY = accuracy_score(y_test, y_pred1)
test_end_time = datetime.now()
TEST_TIMING = test_end_time - test_start_time
print('testing time(HH:MM:SS:ms) - {}\n\n'.format(TEST_TIMING))
#Results
print("Decision Tree Results")
print("Decision Tree Training Accuracy: ",TRAIN_ACCURACY*100,"%")
print("Decision Tree Testing Accuracy: ",TESTING_ACCURACY*100,"%")

print('\n')
print('| Confusion Matrix |')
print('\n')
print(confusion_matrix(y_test, y_pred1))

print('\n')
print(' Classifiction Report ')
print('\n')
classification_report = metrics.classification_report(y_test, y_pred1)
print(classification_report)

training the model..
Done
training_time(HH:MM:SS.ms) - 0:00:00.106179


Predicting test data
testing time(HH:MM:SS:ms) - 0:00:00.002124


Decision Tree Results
Decision Tree Training Accuracy:  99.2665474060823 %
Decision Tree Testing Accuracy:  84.19170243204577 %


| Confusion Matrix |


[[1165  215]
 [ 227 1189]]


 Classifiction Report 


              precision    recall  f1-score   support

           0       0.84      0.84      0.84      1380
           1       0.85      0.84      0.84      1416

    accuracy                           0.84      2796
   macro avg       0.84      0.84      0.84      2796
weighted avg       0.84      0.84      0.84      2796



## RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
train_start_time = datetime.now()
print('training the model..')
model1 = RandomForestClassifier(n_estimators=70)
model1.fit(x_train, y_train)
TRAIN_ACCURACY= accuracy_score(y_train, model1.predict(x_train))
print('Done')
train_end_time = datetime.now()
TIMING =  train_end_time - train_start_time
print('training_time(HH:MM:SS.ms) - {}\n\n'.format(TIMING))
#predicting the traget variable using testing variables
test_start_time = datetime.now()
# predict test data
print('Predicting test data')
y_pred1 = model1.predict(x_test)
TESTING_ACCURACY = accuracy_score(y_test, y_pred1)
test_end_time = datetime.now()
TEST_TIMING = test_end_time - test_start_time
print('testing time(HH:MM:SS:ms) - {}\n\n'.format(TEST_TIMING))
#Results
print("Random forest Results")
print("Random forest Training Accuracy: ",TRAIN_ACCURACY*100,"%")
print("Random forest Testing Accuracy: ",TESTING_ACCURACY*100,"%")

print('\n')
print('| Confusion Matrix |')
print('\n')
print(confusion_matrix(y_test, y_pred1))

print('\n')
print(' Classifiction Report ')
print('\n')
classification_report = metrics.classification_report(y_test, y_pred1)
print(classification_report)

training the model..
Done
training_time(HH:MM:SS.ms) - 0:00:01.537359


Predicting test data
testing time(HH:MM:SS:ms) - 0:00:00.038368


Random forest Results
Random forest Training Accuracy:  100.0 %
Random forest Testing Accuracy:  87.6967095851216 %


| Confusion Matrix |


[[1243  137]
 [ 207 1209]]


 Classifiction Report 


              precision    recall  f1-score   support

           0       0.86      0.90      0.88      1380
           1       0.90      0.85      0.88      1416

    accuracy                           0.88      2796
   macro avg       0.88      0.88      0.88      2796
weighted avg       0.88      0.88      0.88      2796



## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
train_start_time = datetime.now()
print('training the model..')
log_knn = KNeighborsClassifier(n_neighbors=10)
log_knn.fit(x_train, y_train)
TRAIN_ACCURACY= accuracy_score(y_train, log_knn.predict(x_train))
print('Done')
train_end_time = datetime.now()
TIMING =  train_end_time - train_start_time
print('training_time(HH:MM:SS.ms) - {}\n\n'.format(TIMING))

test_start_time = datetime.now()
# predict test data
print('Predicting test data')
y_pred1 = log_knn.predict(x_test)
TESTING_ACCURACY = log_knn.score(x_test,y_test)
test_end_time = datetime.now()
TEST_TIMING = test_end_time - test_start_time
print('testing time(HH:MM:SS:ms) - {}\n\n'.format(TEST_TIMING))


print("KNN Results")
print("KNN Training Accuracy: ",TRAIN_ACCURACY*100,"%")
accuracy = log_knn.score(x_test,y_test)
print("KNN Testing Accuracy: ",TESTING_ACCURACY*100,"%")

print('\n')
print('| Confusion Matrix |')
print('\n')
print(confusion_matrix(y_test, y_pred1))

print('\n')
print(' Classifiction Report ')
print('\n')
classification_report = metrics.classification_report(y_test, y_pred1)
print(classification_report)

training the model..
Done
training_time(HH:MM:SS.ms) - 0:00:01.068877


Predicting test data
testing time(HH:MM:SS:ms) - 0:00:00.429523


KNN Results
KNN Training Accuracy:  89.91949910554563 %
KNN Testing Accuracy:  88.01859799713877 %


| Confusion Matrix |


[[1256  124]
 [ 211 1205]]


 Classifiction Report 


              precision    recall  f1-score   support

           0       0.86      0.91      0.88      1380
           1       0.91      0.85      0.88      1416

    accuracy                           0.88      2796
   macro avg       0.88      0.88      0.88      2796
weighted avg       0.88      0.88      0.88      2796



## SVM

In [ ]:
from sklearn.svm import SVC
SVM_helmet = SVC(kernel='rbf')
helmet_svm = SVM_helmet.fit(x_train, y_train)

RBF

In [ ]:
train_start_time = datetime.now()
print('training the model..')
model= SVC( C=10 ,gamma=1, kernel='rbf' ,decision_function_shape='ovr')
model.fit(x_train,y_train)
TRAIN_ACCURACY= accuracy_score(y_train, model.predict(x_train))
print('Done')
train_end_time = datetime.now()
TIMING =  train_end_time - train_start_time
print('training_time(HH:MM:SS.ms) - {}\n\n'.format(TIMING))

test_start_time = datetime.now()
# predict test data
print('Predicting test data')
y_pred1 = model.predict(x_test)
TESTING_ACCURACY = model.score(x_test,y_test)
test_end_time = datetime.now()
TEST_TIMING = test_end_time - test_start_time
print('testing time(HH:MM:SS:ms) - {}\n\n'.format(TEST_TIMING))
#accuracy on testing data (Unknown data)
#Results
print("SVM OVR RBF Results")
print("SVM OVR RBF Training Accuracy: ",TRAIN_ACCURACY*100,"%")
# accuracy = model.score(x_test,y_test)
print("SVM OVR RBF Testing Accuracy: ",TESTING_ACCURACY*100,"%")

print('\n')
print('| Confusion Matrix |')
print('\n')
print(confusion_matrix(y_test, y_pred1))

print('\n')
print(' Classifiction Report ')
print('\n')
classification_report = metrics.classification_report(y_test, y_pred1)
print(classification_report)

pick = open(r'C:\Users\Sandesh\Downloads\CV\CP\SVM_OVR_RBF.sav', 'wb')
pickle.dump(model ,pick)
pick.close()

training the model..
Done
training_time(HH:MM:SS.ms) - 0:00:05.006817


Predicting test data
testing time(HH:MM:SS:ms) - 0:00:00.919661


SVM OVR RBF Results
SVM OVR RBF Training Accuracy:  91.12701252236135 %
SVM OVR RBF Testing Accuracy:  88.6623748211731 %


| Confusion Matrix |


[[1254  126]
 [ 191 1225]]


 Classifiction Report 


              precision    recall  f1-score   support

           0       0.87      0.91      0.89      1380
           1       0.91      0.87      0.89      1416

    accuracy                           0.89      2796
   macro avg       0.89      0.89      0.89      2796
weighted avg       0.89      0.89      0.89      2796



Linear

In [ ]:
train_start_time = datetime.now()
print('training the model..')
model= SVC( C=10 ,gamma=1, kernel='linear' ,decision_function_shape='ovr')
model.fit(x_train,y_train)
TRAIN_ACCURACY= accuracy_score(y_train, model.predict(x_train))
print('Done')
train_end_time = datetime.now()
TIMING =  train_end_time - train_start_time
print('training_time(HH:MM:SS.ms) - {}\n\n'.format(TIMING))

test_start_time = datetime.now()
# predict test data
print('Predicting test data')
y_pred1 = model.predict(x_test)
TESTING_ACCURACY = model.score(x_test,y_test)
test_end_time = datetime.now()
TEST_TIMING = test_end_time - test_start_time
print('testing time(HH:MM:SS:ms) - {}\n\n'.format(TEST_TIMING))

print("SVM OVR LINEAR Results")
print("SVM OVR LINEAR Training Accuracy: ",TRAIN_ACCURACY*100,"%")

y_pred1 = model.predict(x_test)
accuracy = model.score(x_test,y_test)
print("SVM OVR LINEAR Testing Accuracy: ",TESTING_ACCURACY*100,"%")

print('\n')
print('| Confusion Matrix |')
print('\n')
print(confusion_matrix(y_test, y_pred1))

print('\n')
print(' Classifiction Report ')
print('\n')
classification_report = metrics.classification_report(y_test, y_pred1)
print(classification_report)

training the model..
Done
training_time(HH:MM:SS.ms) - 0:00:10.253894


Predicting test data
testing time(HH:MM:SS:ms) - 0:00:00.748976


SVM OVR LINEAR Results
SVM OVR LINEAR Training Accuracy:  85.49194991055457 %
SVM OVR LINEAR Testing Accuracy:  84.44206008583691 %


| Confusion Matrix |


[[1235  145]
 [ 290 1126]]


 Classifiction Report 


              precision    recall  f1-score   support

           0       0.81      0.89      0.85      1380
           1       0.89      0.80      0.84      1416

    accuracy                           0.84      2796
   macro avg       0.85      0.85      0.84      2796
weighted avg       0.85      0.84      0.84      2796



Polynomial

In [ ]:
train_start_time = datetime.now()
print('training the model..')
model= SVC( C=10 ,gamma=1, kernel='poly' ,decision_function_shape='ovr')
model.fit(x_train,y_train)
TRAIN_ACCURACY= accuracy_score(y_train, model.predict(x_train))
print('Done')
train_end_time = datetime.now()
TIMING =  train_end_time - train_start_time
print('training_time(HH:MM:SS.ms) - {}\n\n'.format(TIMING))

test_start_time = datetime.now()
# predict test data
print('Predicting test data')
y_pred1 = model.predict(x_test)
TESTING_ACCURACY = model.score(x_test,y_test)
test_end_time = datetime.now()
TEST_TIMING = test_end_time - test_start_time
print('testing time(HH:MM:SS:ms) - {}\n\n'.format(TEST_TIMING))

print("SVM OVR POLYNOMIAL Results")
print("SVM OVR POLYNOMIAL Training Accuracy: ",TRAIN_ACCURACY*100,"%")

y_pred1 = model.predict(x_test)
accuracy = model.score(x_test,y_test)
print("SVM OVR POLYNOMIAL Testing Accuracy: ",TESTING_ACCURACY*100,"%")

print('\n')
print('| Confusion Matrix |')
print('\n')
print(confusion_matrix(y_test, y_pred1))

print('\n')
print(' Classifiction Report ')
print('\n')
classification_report = metrics.classification_report(y_test, y_pred1)
print(classification_report)

training the model..
Done
training_time(HH:MM:SS.ms) - 0:07:08.333520


Predicting test data
testing time(HH:MM:SS:ms) - 0:00:00.560621


SVM OVR POLYNOMIAL Results
SVM OVR POLYNOMIAL Training Accuracy:  84.99105545617174 %
SVM OVR POLYNOMIAL Testing Accuracy:  85.33619456366237 %


| Confusion Matrix |


[[1109  271]
 [ 139 1277]]


 Classifiction Report 


              precision    recall  f1-score   support

           0       0.89      0.80      0.84      1380
           1       0.82      0.90      0.86      1416

    accuracy                           0.85      2796
   macro avg       0.86      0.85      0.85      2796
weighted avg       0.86      0.85      0.85      2796



Sigmoid

In [ ]:
train_start_time = datetime.now()
print('training the model..')
model= SVC( C=10 ,gamma=1, kernel='sigmoid' ,decision_function_shape='ovr')
model.fit(x_train,y_train)
TRAIN_ACCURACY= accuracy_score(y_train, model.predict(x_train))
print('Done')
train_end_time = datetime.now()
TIMING =  train_end_time - train_start_time
print('training_time(HH:MM:SS.ms) - {}\n\n'.format(TIMING))

test_start_time = datetime.now()
# predict test data
print('Predicting test data')
y_pred1 = model.predict(x_test)
TESTING_ACCURACY = model.score(x_test,y_test)
test_end_time = datetime.now()
TEST_TIMING = test_end_time - test_start_time
print('testing time(HH:MM:SS:ms) - {}\n\n'.format(TEST_TIMING))

print("sigmoid Results")
print("sigmoid Training Accuracy: ",TRAIN_ACCURACY*100,"%")

y_pred1 = model.predict(x_test)
accuracy = model.score(x_test,y_test)
print("sigmoid Testing Accuracy: ",TESTING_ACCURACY*100,"%")

print('\n')
print('| Confusion Matrix |')
print('\n')
print(confusion_matrix(y_test, y_pred1))

print('\n')
print(' Classifiction Report ')
print('\n')
classification_report = metrics.classification_report(y_test, y_pred1)
print(classification_report)

training the model..
Done
training_time(HH:MM:SS.ms) - 0:00:08.135966


Predicting test data
testing time(HH:MM:SS:ms) - 0:00:01.511437


sigmoid Results
sigmoid Training Accuracy:  59.83899821109123 %
sigmoid Testing Accuracy:  59.620886981402 %


| Confusion Matrix |


[[832 548]
 [581 835]]


 Classifiction Report 


              precision    recall  f1-score   support

           0       0.59      0.60      0.60      1380
           1       0.60      0.59      0.60      1416

    accuracy                           0.60      2796
   macro avg       0.60      0.60      0.60      2796
weighted avg       0.60      0.60      0.60      2796



## GNB

In [ ]:
from sklearn.naive_bayes import GaussianNB


In [ ]:
train_start_time = datetime.now()
print('training the model..')
model1 = GaussianNB()
model1.fit(x_train, y_train)
TRAIN_ACCURACY= accuracy_score(y_train, model1.predict(x_train))
print('Done')
train_end_time = datetime.now()
TIMING =  train_end_time - train_start_time
print('training_time(HH:MM:SS.ms) - {}\n\n'.format(TIMING))
#predicting the traget variable using testing variables
test_start_time = datetime.now()
# predict test data
print('Predicting test data')
y_pred1 = model1.predict(x_test)
TESTING_ACCURACY = accuracy_score(y_test, y_pred1)
test_end_time = datetime.now()
TEST_TIMING = test_end_time - test_start_time
print('testing time(HH:MM:SS:ms) - {}\n\n'.format(TEST_TIMING))
#Results
print("GaussianNB Results")
print("GaussianNB Training Accuracy: ",TRAIN_ACCURACY*100,"%")
print("GaussianNB Testing Accuracy: ",TESTING_ACCURACY*100,"%")

print('\n')
print('| Confusion Matrix |')
print('\n')
print(confusion_matrix(y_test, y_pred1))

print('\n')
print(' Classifiction Report ')
print('\n')
classification_report = metrics.classification_report(y_test, y_pred1)
print(classification_report)

training the model..
Done
training_time(HH:MM:SS.ms) - 0:00:00.008114


Predicting test data
testing time(HH:MM:SS:ms) - 0:00:00.002600


GaussianNB Results
GaussianNB Training Accuracy:  85.5098389982111 %
GaussianNB Testing Accuracy:  84.08440629470672 %


| Confusion Matrix |


[[1213  167]
 [ 278 1138]]


 Classifiction Report 


              precision    recall  f1-score   support

           0       0.81      0.88      0.85      1380
           1       0.87      0.80      0.84      1416

    accuracy                           0.84      2796
   macro avg       0.84      0.84      0.84      2796
weighted avg       0.84      0.84      0.84      2796



## AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
train_start_time = datetime.now()
print('training the model..')
model1 = AdaBoostClassifier(random_state = 96)
model1.fit(x_train, y_train)
TRAIN_ACCURACY= accuracy_score(y_train, model1.predict(x_train))
print('Done')
train_end_time = datetime.now()
TIMING =  train_end_time - train_start_time
print('training_time(HH:MM:SS.ms) - {}\n\n'.format(TIMING))
#predicting the traget variable using testing variables
test_start_time = datetime.now()
# predict test data
print('Predicting test data')
y_pred1 = model1.predict(x_test)
TESTING_ACCURACY = accuracy_score(y_test, y_pred1)
test_end_time = datetime.now()
TEST_TIMING = test_end_time - test_start_time
print('testing time(HH:MM:SS:ms) - {}\n\n'.format(TEST_TIMING))
#Results
print("AdaBoost Results")
print("AdaBoost Training Accuracy: ",TRAIN_ACCURACY*100,"%")
print("AdaBoost Testing Accuracy: ",TESTING_ACCURACY*100,"%")

print('\n')
print('| Confusion Matrix |')
print('\n')
print(confusion_matrix(y_test, y_pred1))

print('\n')
print(' Classifiction Report ')
print('\n')
classification_report = metrics.classification_report(y_test, y_pred1)
print(classification_report)

training the model..
Done
training_time(HH:MM:SS.ms) - 0:00:00.626138


Predicting test data
testing time(HH:MM:SS:ms) - 0:00:00.024656


AdaBoost Results
AdaBoost Training Accuracy:  85.82289803220036 %
AdaBoost Testing Accuracy:  84.97854077253218 %


| Confusion Matrix |


[[1197  183]
 [ 237 1179]]


 Classifiction Report 


              precision    recall  f1-score   support

           0       0.83      0.87      0.85      1380
           1       0.87      0.83      0.85      1416

    accuracy                           0.85      2796
   macro avg       0.85      0.85      0.85      2796
weighted avg       0.85      0.85      0.85      2796

